In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly as py
import datetime
import seaborn as sns
from google.colab import drive
from sklearn import preprocessing
py.offline.init_notebook_mode(connected=True)
import tensorflow as tf
from tensorflow import keras
from keras.layers import Flatten
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import GlobalAveragePooling1D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import LeakyReLU
from keras import optimizers
from keras.layers import Dense, Activation, Dropout
from keras.layers import AveragePooling1D
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.layers import LSTM
from keras.layers import Reshape
from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support
from matplotlib.pyplot import figure
from keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
import gc

In [5]:



drive.mount('/content/drive/')
aapl = pd.read_csv("/content/drive/My Drive/Stock Price Prediction Research/Data/aapl_prices_fiveminute_stats_wsh.csv")
aapl.head(5)

Mounted at /content/drive/


,Unnamed: 0,PRICE_MEAN,VOLUME_MEAN,PRICE_HIGH,PRICE_LOW,PRICE_OPEN,PRICE_CLOSE
0,1/2/2001 9:30,14.930692,565.354331,15.0000,14.8750,14.8750,14.8750
1,1/2/2001 9:35,14.936993,707.792208,15.0625,14.6875,14.8750,14.9375
2,1/2/2001 9:40,15.021382,944.939271,15.0625,14.9375,14.9375,15.0625
3,1/2/2001 9:45,15.114291,735.205992,15.2500,15.0000,15.0625,15.0625
4,1/2/2001 9:50,14.938517,1075.739645,15.0625,14.8125,15.0625,14.9375


In [7]:
date_time = pd.to_datetime(aapl['Unnamed: 0'], infer_datetime_format=True)
datetime_table = pd.DataFrame({'DAY':date_time.dt.day, 'MONTH':date_time.dt.month,'YEAR':date_time.dt.year,'HOUR': date_time.dt.hour, 'MINUTE': date_time.dt.minute})
aapl = aapl.join(datetime_table, how = 'outer')

ValueError: ignored

In [ ]:
aapl

In [8]:
def stocksplit(stock,day, month, year, ratio):
  date_index = stock.loc[(stock['YEAR'] == year) & (stock['DAY'] == day) & (stock['MONTH']==month)].index.values.astype(int)[0]
  pre_split_high  = stock.iloc[:date_index]['PRICE_HIGH']
  pre_split_low = stock.iloc[:date_index]['PRICE_LOW']
  pre_split_open = stock.iloc[:date_index]['PRICE_OPEN']
  pre_split_close = stock.iloc[:date_index]['PRICE_CLOSE']
  pre_split_mean = stock.iloc[:date_index]['PRICE_MEAN']
  pre_split_volume = stock.iloc[:date_index]['VOLUME_MEAN']

  print(pre_split_volume)
  post_split_high  = stock.iloc[date_index:]['PRICE_HIGH']
  post_split_low = stock.iloc[date_index:]['PRICE_LOW']
  post_split_open = stock.iloc[date_index:]['PRICE_OPEN']
  post_split_close = stock.iloc[date_index:]['PRICE_CLOSE']
  post_split_mean = stock.iloc[date_index:]['PRICE_MEAN']
  post_split_volume = stock.iloc[date_index:]['VOLUME_MEAN']
  print(post_split_volume)
  post_split_high  = post_split_high*2
  post_split_low = post_split_low*2
  post_split_open = post_split_open*2
  post_split_close = post_split_close*2
  post_split_mean = post_split_mean*2
  post_split_volume = post_split_volume/2
  
  print(post_split_volume)
  
  price_high  = pre_split_high.append(post_split_high)
  price_low = pre_split_low.append(post_split_low)
  price_open = pre_split_open.append(post_split_open)
  price_close = pre_split_close.append(post_split_close)
  price_mean = pre_split_mean.append(post_split_mean)
  volume = pre_split_volume.append(post_split_volume)
  stock["PRICE_HIGH"] = price_high
  stock["PRICE_LOW"] = price_low
  stock["PRICE_OPEN"] = price_open
  stock["PRICE_CLOSE"] = price_close
  stock["PRICE_MEAN"] = price_mean
  stock["VOLUME_MEAN"] = volume
  return stock

In [9]:
aapl = stocksplit(aapl, 28, 2, 2005, 2)
aapl

0         565.354331
1         707.792208
2         944.939271
3         735.205992
4        1075.739645
            ...     
81972     272.685892
81973     311.289003
81974     276.734310
81975     304.983713
81976    1514.884615
Name: VOLUME_MEAN, Length: 81977, dtype: float64
81977       310.774035
81978       260.109650
81979       266.699053
81980       249.828864
81981       278.068268
              ...     
198227      189.182561
198228      201.234844
198229      151.116947
198230      225.504791
198231    14462.520000
Name: VOLUME_MEAN, Length: 116255, dtype: float64
81977      155.387018
81978      130.054825
81979      133.349527
81980      124.914432
81981      139.034134
             ...     
198227      94.591281
198228     100.617422
198229      75.558473
198230     112.752395
198231    7231.260000
Name: VOLUME_MEAN, Length: 116255, dtype: float64


,Unnamed: 0,PRICE_MEAN,VOLUME_MEAN,PRICE_HIGH,PRICE_LOW,PRICE_OPEN,PRICE_CLOSE,DAY,MONTH,YEAR,HOUR,MINUTE
0,1/2/2001 9:30,14.930692,565.354331,15.0000,14.8750,14.8750,14.8750,2,1,2001,9,30
1,1/2/2001 9:35,14.936993,707.792208,15.0625,14.6875,14.8750,14.9375,2,1,2001,9,35
2,1/2/2001 9:40,15.021382,944.939271,15.0625,14.9375,14.9375,15.0625,2,1,2001,9,40
3,1/2/2001 9:45,15.114291,735.205992,15.2500,15.0000,15.0625,15.0625,2,1,2001,9,45
4,1/2/2001 9:50,14.938517,1075.739645,15.0625,14.8125,15.0625,14.9375,2,1,2001,9,50
...,...,...,...,...,...,...,...,...,...,...,...,...
198227,12/31/2010 15:40,644.036210,94.591281,644.1200,643.9000,644.0000,643.9600,31,12,2010,15,40
198228,12/31/2010 15:45,643.106627,100.617422,643.9600,642.6200,643.9000,642.9000,31,12,2010,15,45
198229,12/31/2010 15:50,643.571344,75.558473,644.2200,642.8800,642.9000,644.1160,31,12,2010,15,50
198230,12/31/2010 15:55,644.721545,112.752395,645.4000,644.0200,644.0600,645.3000,31,12,2010,15,55


# **FEATURE** **ENGINEERING**


In [10]:
def diff(df, col):
  orig_df = df.copy()
  difference = pd.DataFrame({'DIFF_'+col:np.diff(df[col])})
  orig_df = orig_df.join(difference, how = 'outer')
  return orig_df
  

In [11]:
aapl_diff = diff(aapl, 'PRICE_OPEN')
aapl_diff = diff(aapl_diff, 'PRICE_CLOSE')
aapl_diff = diff(aapl_diff, 'PRICE_HIGH')
aapl_diff = diff(aapl_diff, 'PRICE_LOW')
aapl_diff = diff(aapl_diff, 'PRICE_MEAN')
aapl_diff = diff(aapl_diff, 'VOLUME_MEAN')



In [12]:
def perc_change(df, col):
  original = df.loc[:,col]
  difference = df.loc[:, 'DIFF_'+ col]
  percentage = difference/original
  perc_df = pd.DataFrame({'PERC_'+col: percentage})
  df = df.join(perc_df, how = 'outer')
  return df


In [13]:
aapl_diff = perc_change(aapl_diff, 'PRICE_OPEN')
aapl_diff = perc_change(aapl_diff, 'PRICE_CLOSE')

In [14]:
def range_func(df):
  high = df.PRICE_HIGH
  low = df.PRICE_LOW
  hl_range = high-low 
  difference = np.diff(hl_range)
  diff = pd.DataFrame({'RANGE':difference})
  df=df.join(diff, how= 'outer')
  return df

In [15]:
aapl_diff = range_func(aapl_diff)

In [16]:
def time_of_day(df):
  orig_df = df.copy()
  hour = orig_df.HOUR 
  part_of_day = [1 if x < 11 else 2 if x<13 else 3 for x in hour]
  part_of_day = pd.DataFrame({'TIME':part_of_day})
  orig_df = orig_df.join(part_of_day,how = 'outer')
  return orig_df

In [17]:
aapl_diff = time_of_day(aapl_diff)

In [18]:
time_of_day = pd.get_dummies(aapl_diff.TIME)
time_of_day = time_of_day.rename(columns={1:'MORNING',2:'AFTERNOON', 3:'EVENING'})
aapl_diff = aapl_diff.join(time_of_day, how = 'outer')

In [19]:
def day_of_week(df):
  orig_df = df.copy()
  date = [datetime.date(y,m,d) for y,m,d in zip(orig_df.YEAR, orig_df.MONTH, orig_df.DAY)]
  part_of_week = [x.weekday() for x in date]
  part_of_week = pd.DataFrame({'DAY_OF_WEEK': part_of_week})
  orig_df = orig_df.join(part_of_week, how = 'outer')
  return orig_df

In [20]:
aapl_diff = day_of_week(aapl_diff)

In [21]:
day_of_week = pd.get_dummies(aapl_diff.DAY_OF_WEEK)
day_of_week = day_of_week.rename(columns={0:'MON',1:'TUE', 2:'WED',3:'THUR',4:'FRI'})
aapl_diff = aapl_diff.join(day_of_week, how = 'outer')

In [22]:
def movement(df):
  orig_df = df.copy()
  movement= [0 if x< 0 else 1 for x in orig_df['DIFF_PRICE_OPEN']]
  movement = pd.DataFrame({'MOVEMENT': movement})
  
  return movement

    

In [23]:
def n_timesteps(df, n, feature):
  feature_df = df[[feature]]
  #temp = (feature_df - feature_df.min()) /(feature_df.max()-feature_df.min())
  temp = feature_df
  for i in range(1,n+1):
    shifted  = temp.shift(i).loc[:,feature]
    feature_df = feature_df.join(shifted, how = 'outer', rsuffix = i)

  return feature_df[n:-1]


In [24]:
n = 2
df_open = n_timesteps(aapl_diff, n,'DIFF_PRICE_OPEN')
df_close = n_timesteps(aapl_diff,n, 'DIFF_PRICE_CLOSE')
df_high = n_timesteps(aapl_diff,n, 'DIFF_PRICE_HIGH')
df_low = n_timesteps(aapl_diff,n, 'DIFF_PRICE_LOW')
df_mean = n_timesteps(aapl_diff,n, 'DIFF_PRICE_MEAN')
df_vol = n_timesteps(aapl_diff,n, 'DIFF_VOLUME_MEAN')
df_open_perc = n_timesteps(aapl_diff, n, 'PERC_PRICE_OPEN')
df_close_perc = n_timesteps(aapl_diff, n, 'PERC_PRICE_CLOSE')
df_morning = n_timesteps(aapl_diff,n, 'MORNING')
df_afternoon = n_timesteps(aapl_diff, n, 'AFTERNOON')
df_evening = n_timesteps(aapl_diff, n, 'EVENING')
df_mon = n_timesteps(aapl_diff,n, 'MON')
df_tue = n_timesteps(aapl_diff,n, 'TUE')
df_wed = n_timesteps(aapl_diff,n, 'WED')
df_thur = n_timesteps(aapl_diff,n, 'THUR')
df_fri = n_timesteps(aapl_diff,n, 'FRI')
df_range = n_timesteps(aapl_diff, n,'RANGE')
df_month = n_timesteps(aapl_diff, n, 'MONTH')
df_day = n_timesteps(aapl_diff,n,'DAY')
aapl_diff.head(5)

,Unnamed: 0,PRICE_MEAN,VOLUME_MEAN,PRICE_HIGH,PRICE_LOW,PRICE_OPEN,PRICE_CLOSE,DAY,MONTH,YEAR,HOUR,MINUTE,DIFF_PRICE_OPEN,DIFF_PRICE_CLOSE,DIFF_PRICE_HIGH,DIFF_PRICE_LOW,DIFF_PRICE_MEAN,DIFF_VOLUME_MEAN,PERC_PRICE_OPEN,PERC_PRICE_CLOSE,RANGE,TIME,MORNING,AFTERNOON,EVENING,DAY_OF_WEEK,MON,TUE,WED,THUR,FRI
0,1/2/2001 9:30,14.930692,565.354331,15.0000,14.8750,14.8750,14.8750,2,1,2001,9,30,0.0000,0.0625,0.0625,-0.1875,0.006300,142.437877,0.000000,0.004202,0.2500,1,1,0,0,1,0,1,0,0,0
1,1/2/2001 9:35,14.936993,707.792208,15.0625,14.6875,14.8750,14.9375,2,1,2001,9,35,0.0625,0.1250,0.0000,0.2500,0.084389,237.147063,0.004202,0.008368,-0.2500,1,1,0,0,1,0,1,0,0,0
2,1/2/2001 9:40,15.021382,944.939271,15.0625,14.9375,14.9375,15.0625,2,1,2001,9,40,0.1250,0.0000,0.1875,0.0625,0.092909,-209.733279,0.008368,0.000000,0.1250,1,1,0,0,1,0,1,0,0,0
3,1/2/2001 9:45,15.114291,735.205992,15.2500,15.0000,15.0625,15.0625,2,1,2001,9,45,0.0000,-0.1250,-0.1875,-0.1875,-0.175774,340.533652,0.000000,-0.008299,0.0000,1,1,0,0,1,0,1,0,0,0
4,1/2/2001 9:50,14.938517,1075.739645,15.0625,14.8125,15.0625,14.9375,2,1,2001,9,50,-0.1875,-0.1250,-0.1250,-0.0625,-0.104371,-599.592856,-0.012448,-0.008368,-0.0625,1,1,0,0,1,0,1,0,0,0


In [25]:
df_open

,DIFF_PRICE_OPEN,DIFF_PRICE_OPEN1,DIFF_PRICE_OPEN2
2,0.1250,0.0625,0.0000
3,0.0000,0.1250,0.0625
4,-0.1875,0.0000,0.1250
5,-0.1250,-0.1875,0.0000
6,0.0625,-0.1250,-0.1875
...,...,...,...
198226,0.1000,-0.0400,-0.1600
198227,-0.1000,0.1000,-0.0400
198228,-1.0000,-0.1000,0.1000
198229,1.1600,-1.0000,-0.1000


In [26]:
#This cell converts all DataFrames with n lags into numpy arrays.


array_open = df_open.iloc[:, 1:].to_numpy()
array_close = df_close.iloc[:, 1:].to_numpy()
array_high = df_high.iloc[:, 1:].to_numpy()
array_low =df_low.iloc[:, 1:].to_numpy()
array_mean = df_mean.iloc[:, 1:].to_numpy()
array_vol = df_vol.iloc[:, 1:].to_numpy()
array_open_perc = df_open_perc.iloc[:, 1:].to_numpy()
array_close_perc = df_close_perc.iloc[:, 1:].to_numpy()
array_morning = df_morning.iloc[:, 1:].to_numpy()
array_afternoon =df_afternoon.iloc[:, 1:].to_numpy()
array_evening = df_evening.iloc[:, 1:].to_numpy()
array_mon = df_mon.iloc[:, 1:].to_numpy()
array_tue = df_tue.iloc[:, 1:].to_numpy()
array_wed = df_wed.iloc[:, 1:].to_numpy()
array_thur = df_thur.iloc[:, 1:].to_numpy()
array_fri = df_fri.iloc[:, 1:].to_numpy()
array_range = df_range.iloc[:, 1:].to_numpy()
array_month = df_month.iloc[:, 1:].to_numpy()
array_day = df_day.iloc[:, 1:].to_numpy()

In [27]:
x_val = np.stack((array_open, array_close,array_high, array_low, array_mean, array_vol, array_open_perc, array_close_perc, array_mon,array_tue,array_wed,array_thur,array_fri,array_morning,array_afternoon, array_evening, array_range, array_month,array_day), axis=1)

In [28]:
y_val = movement(df_open[['DIFF_PRICE_OPEN']]).to_numpy()
y_val[:10]

array([[1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1]])

In [29]:
array_open = 0
array_close = 0
array_high = 0
array_low =0
array_mean = 0
array_vol = 0
array_open_perc = 0
array_close_perc = 0
array_time_of_day = 0
array_day_of_week = 0
array_range =0
array_month = 0
array_day = 0

df_open = 0
df_close = 0
df_high = 0
df_low = 0
df_mean = 0
df_vol =0
df_open_perc = 0
df_close_perc = 0
df_time_of_day =0
df_day_of_week = 0
df_range = 0
df_month = 0
df_day = 0

In [30]:
gc.collect()

231

In [31]:
"" #This cell converts the non-lagged data into training array and test array.

#y_val = aapl_diff[['MOVEMENT']].to_numpy()[1:]
# x_val = aapl_diff[['MONTH','DAY','DAY_OF_WEEK','TIME','PERC_PRICE_OPEN','PERC_PRICE_CLOSE','DIFF_VOLUME_MEAN','DIFF_PRICE_MEAN','DIFF_PRICE_HIGH','DIFF_PRICE_LOW','RANGE']]
# x_val = (x_val- x_val.min())/(x_val.max()-x_val.min())
# x_val = x_val.to_numpy()[:-1]
# samples = list()
# for i in range(0,x_val.shape[0]-20000):
# 	# grab from i to i + 200
# 	sample = x_val[i:i+20000]
# 	samples.append(sample)
# samples = np.array(samples)
# len(samples)

''

# Differencing per Day


In [32]:

# movement_lag_data = n_lag_steps_diff(aapl, 80)
# movement_onlylags = movement_lag_data.copy().drop(columns = 'PRICE')

# movement_onlyprice = (movement_lag_data[['PRICE']])
# movement_onlylags = movement_onlylags.to_numpy()
# movement_onlyprice = movement_onlyprice.to_numpy()
# print(movement_onlylags.shape[0])
# movement_trainX = movement_onlylags[:round(len(movement_lag_data)*.8)]
# movement_trainY = movement_onlyprice[:round(len(movement_onlyprice)*.8)]
# movement_testX = movement_onlylags[round(len(movement_lag_data)*.8):]
# movement_testY = movement_onlyprice[round(len(movement_onlyprice)*.8):]
# movement_onlylags.reshape(movement_onlylags.shape[0], movement_onlylags.shape[1], 1)
# print(movement_onlylags.shape)
# movement_onlylags.reshape(movement_onlylags.shape[0],1, movement_onlylags.shape[1])
# movement_onlylags.shape[0]

# Differencing per Min


In [33]:
# movement_lags_min = movement(aapl_min)
# movement_lag_data_min = n_lag_steps_diff(aapl_min, 10)
# movement_onlylags_min = movement_lag_data_min[['LAG1','LAG2','LAG3','LAG4','LAG5','LAG6','LAG7','LAG8','LAG9','LAG10']]
# movement_onlyprice_min = movement_lag_data_min[['PRICE']]
# movement_onlylags_min= movement_onlylags_min.to_numpy()
# movement_onlyprice_min = movement_onlyprice_min.to_numpy()
# movement_trainX_min = movement_onlylags_min[:round(len(movement_lag_data_min)*.8)]
# movement_trainY_min = movement_onlyprice_min[:round(len(movement_onlyprice_min)*.8)]
# movement_testX_min = movement_onlylags_min[round(len(movement_lag_data_min)*.8):]
# movement_testY_min = movement_onlyprice_min[round(len(movement_onlyprice_min)*.8):]
# movement_onlylags_min.reshape(movement_onlylags_min.shape[0], movement_onlylags_min.shape[1], 1)
# movement_lag_data_min

# CNN Architecture

In [56]:
def evaluate_model(trainX, trainy, testX, testy, batch_size, epochs,verbose, model_weights = None):
  #look into automatic keras neural network layer choice
  #SVM
    

      verbose, epochs, batch_size = verbose, epochs, batch_size
      n_timesteps, n_features, n_outputs = 80, 1, 1

      #trainX = trainX.reshape(trainX.shape[0], trainX.shape[1],1)
      #testX = testX.reshape(testX.shape[0], testX.shape[1],1)
      
      model = Sequential()
      model.add(LSTM(40, stateful = True, return_sequences=True,batch_input_shape = (batch_size, trainX.shape[1], trainX.shape[2])))
      
      
      #model.add(Conv1D( filters = 3, kernel_size = 1, activation='relu'))
      #model.add(Reshape((1,1,80,1)))
      #model.add(TimeDistributed(AveragePooling1D()))
      #model.add(TimeDistributed(Conv1D(100, 10, activation='relu')))
      #model.add(TimeDistributed(Flatten()))
      #model.add(LSTM(60, stateful = True, return_sequences=True,batch_input_shape = (batch_size, trainX.shape[1], trainX.shape[2])))
      model.add(LSTM(120, stateful = True, return_sequences=True))
      model.add(LSTM(120, stateful = True, return_sequences=True))
      model.add(LSTM(40, stateful = True))
      #model.add(Dropout(0.4))
      model.add(Dense(50, activation='relu'))
     
      model.add(Dropout(0.4))
      model.add(Dense(1, activation = 'sigmoid'))
      #opt = optimizers.adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1, decay=0.001, amsgrad=False)

             
      if model_weights == True:
        model.load_weights("/content/drive/My Drive/Stock Price Prediction Research/weights.best.hdf5")
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
      else:
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
      # fit network
      filepath="/content/drive/My Drive/Stock Price Prediction Research/weights.best.hdf5"
      checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=0, save_best_only=True, mode='max')
      callbacks_list = [checkpoint]

      model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose, callbacks = callbacks_list, shuffle = False)
      # evaluate model"
      _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
      
      pred = model.predict(testX, batch_size = batch_size, verbose = 0)
      weights = True
      clear_session()
      
      return pred, weights,accuracy

In [55]:
def walk_forward(initial_epochs, window, batch_size, wf_epochs):
  prediction_days = []
  actual_days = []
  walk_train = []
  walk_test = []

  x = x_val
  y = y_val
  # window = 2**13 *16a
  # batch_size = 2 **10
  multiplier = 2 **10
  initial_mult = 2 **12
  #initial_batch_size =2 **10
  accuracy_days = pd.DataFrame(columns=['Minute','Accuracy'])
  p, weights, acc = evaluate_model(x[:window], y[:window], x[window:window+initial_mult], y[window:window+initial_mult], batch_size, initial_epochs, 2)
  print(acc)

  for i in range (1,500):
    trainx = x[i:i+window]
    trainy = y[i:i+window]
    
    
    testx = x[window + i: window + batch_size+i]
    testy = y[window + i: window + batch_size+i]

    pred, weights,acc = evaluate_model(trainx, trainy, testx, testy, batch_size, wf_epochs,2, weights)
    
    if pred[0][0] > .5:
      prediction_days.extend([1])
    else:
      prediction_days.extend([0])
  
    actual_days.extend(testy[0])
  
    # day = pd.DataFrame({'Minute': i*5 +window, 'Accuracy': [acc]})
    # accuracy_days = accuracy_days.append(day,ignore_index=True)
    
    print("Minute"+str(i*5), acc)
    print(pred[0])
    print(testy[0])
  p_r = precision_recall_fscore_support(actual_days,prediction_days,labels = [0,1])
  return prediction_days, p_r, actual_days
    
  

In [36]:

def plot_prediction(actual_days, prediction_days, total, cutoff):
  binary_predict = np.array([1 if x> .5 else -1 for x in actual_days])
  binary_actual = np.array([1 if x==1 else -1 for x in prediction_days])
  #print(binary_actual, prediction_days)
  tot_predict = 0
  tot_actual =0 
  plot_predict = []
  plot_actual =[]
  
  for i in range((len(binary_predict))):
    
    tot_predict += binary_predict[i]
    tot_actual += binary_actual[i]
    plot_predict.append(tot_predict)
    plot_actual.append(tot_actual)
  plot_df = (pd.DataFrame({'Predict' : plot_predict, "Actual" : plot_actual}))

  plot_df.plot(figsize = (10,10))
  plt.legend()
  plt.axvline(x=cutoff)

  plt.show()

In [62]:
pred, fscore, actual = walk_forward(100,2**13, 2**9,100)

Streaming output truncated to the last 5000 lines.
Epoch 63/100
16/16 - 0s - loss: 0.6800 - accuracy: 0.5809
Epoch 64/100
16/16 - 0s - loss: 0.6801 - accuracy: 0.5809
Epoch 65/100
16/16 - 0s - loss: 0.6799 - accuracy: 0.5809
Epoch 66/100
16/16 - 0s - loss: 0.6801 - accuracy: 0.5809
Epoch 67/100
16/16 - 0s - loss: 0.6799 - accuracy: 0.5809
Epoch 68/100
16/16 - 0s - loss: 0.6803 - accuracy: 0.5809
Epoch 69/100
16/16 - 0s - loss: 0.6802 - accuracy: 0.5809
Epoch 70/100
16/16 - 0s - loss: 0.6801 - accuracy: 0.5809
Epoch 71/100
16/16 - 0s - loss: 0.6801 - accuracy: 0.5809
Epoch 72/100
16/16 - 0s - loss: 0.6802 - accuracy: 0.5809
Epoch 73/100
16/16 - 0s - loss: 0.6801 - accuracy: 0.5809
Epoch 74/100
16/16 - 0s - loss: 0.6800 - accuracy: 0.5809
Epoch 75/100
16/16 - 0s - loss: 0.6801 - accuracy: 0.5809
Epoch 76/100
16/16 - 0s - loss: 0.6801 - accuracy: 0.5809
Epoch 77/100
16/16 - 0s - loss: 0.6801 - accuracy: 0.5809
Epoch 78/100
16/16 - 0s - loss: 0.6803 - accuracy: 0.5809
Epoch 79/100
16/16 - 

In [ ]:
#This cell is meant to tune the hyper parameters for the RNN.

epoch_range = [100, 200, 300, 400, 500]
training_window_size = [2**13 *11, 2**13 *12, 2**13 *13, 2**13 *14, 2**13 *15]
wf_epoch_range = [1,3,5,10,20]
batch_size = [2**9,2**10, 2**11, 2**12]
tuning = pd.DataFrame(columns = ['Epoch Size','Training Set Size','WF Epoch Size','Batch Size', 'Precision Down','Precision Up', 'Recall Down', 'Recall Up'])

for i in epoch_range:
  for j in training_window_size:
    for k in wf_epoch_range:
        for b in batch_size:
          prediction, prec_recall, actual_days = walk_forward(i, j, b, k)
          
          p_down = prec_recall[0][0]
          p_up = prec_recall[0][1]
          r_down = prec_recall[1][0]
          r_up = prec_recall[1][1]
          plot_prediction(actual_days, prediction, 0,0)
          row = {'Epoch Size': i,'Training Set Size':j,'WF Epoch Size':k,'Batch Size':b, 'Precision Down':p_down,'Precision Up':p_up, 'Recall Down':r_down, 'Recall Up':r_up}
          tuning = tuning.append(row, ignore_index = True)
tuning          





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100


InvalidArgumentError: ignored

In [ ]:
plot_prediction(actual_days, prediction_days, 0,0)

In [ ]:
def profit(predicted, prices, lags, capital, initial_window, multiplier):
  figure(num=None, figsize=(25, 10), dpi=80, facecolor='w', edgecolor='k')
  money = []
  shares = 0
  last_price = 0
  num_predicted = len(predicted)
  sell = initial_window
  prices = prices[initial_window+ multiplier : initial_window +multiplier + num_predicted]

  for pred, price in zip(predicted, prices):
    num_of_shares = 1

    if pred == 0:
      if shares - num_of_shares <= 0:
         money.append(shares * price + capital)
         sell +=1
         continue
      capital += price *num_of_shares
      shares -= num_of_shares
      print('Sold at '+ str(price))
      plt.axvline(sell)
    else:
      
      if capital < price:
        money.append(shares * price + capital)
        sell +=1
        continue
      plt.axvline(sell, color = 'g')
      capital -= price* num_of_shares
      shares += num_of_shares
      print('Bought at '+ str(price))
    money.append(shares * price + capital)  
    last_price = price
    sell +=1
  
  
  graph= pd.DataFrame({'Total': money,'Price': prices})  
  
  ax1 = graph.Total.plot(figsize = (100,100))
  
  ax2 = ax1.twinx()
  ax2.spines['right'].set_position(('axes', 1.0))
  graph.Price.plot(color = 'r', secondary_y = True)
  plt.legend()
  plt.show()
  return shares * last_price + capital


In [ ]:

profit(prediction_days, aapl['PRICE'], 80, 3000,2000, 10)

In [ ]:
aapl.iloc[1040:1110]

In [ ]:
 
tot_predict = 0
tot_actual = 0

binary_predict = np.array([1 if x[0]> .5 else -1 for x in y_output])
binary_actual = np.array([1 if x[0]==1 else -1 for x in movement_testY])
plot_predict = []
plot_actual =[]
for i in range(len(binary_predict)):
  tot_predict += binary_predict[i]
  tot_actual += binary_actual[i]
  plot_predict.append(tot_predict)
  plot_actual.append(tot_actual)
plot_df = (pd.DataFrame({'Predict' : plot_predict, "Actual" : plot_actual}))
plot_df.plot()
plt.legend()
plt.show()
precision_recall_fscore_support(binary_predict, binary_actual)

In [37]:
y_output_min = evaluate_model(movement_trainX_min, movement_trainY_min, movement_testX_min, movement_testY_min, 5000, 200)

NameError: ignored

In [ ]:
 
tot_predict = 0
tot_actual = 0

binary_predict = np.array([1 if x[0]> .5 else 0 for x in y_output_min])
binary_actual = np.array([1 if x[0]==1 else 0 for x in movement_testY_min])
plot_predict = []
plot_actual =[]
for i in range(len(binary_predict)):
  tot_predict += binary_predict[i]
  tot_actual += binary_actual[i]
  plot_predict.append(tot_predict)
  plot_actual.append(tot_actual)
plot_df = (pd.DataFrame({'Predict' : plot_predict, "Actual" : plot_actual}))
plot_df.plot()
plt.legend()
plt.show()



In [ ]:
precision_recall_fscore_support(binary_predict, binary_actual)


In [ ]:
movement_testY